# Sect 38: Time Series Models

- 10/23/20
- online-ds-pt-041320

## Learning Objectives:

- Revisit types of time series trends and how to remove them.
- Revisit seasonal decomposition`statsmodels.tsa.seasonal.seasonal_decompose`

- Learn about PACF, ACF
- Introduce ARIMA and SARIMA models.
- Activity: SARIMA Models - Lab




## Questions

# Reviewing the End of Sect 37

In [ ]:
from fsds.imports import *
pd.set_option('precision',3)
plt.rcParams['figure.figsize'] = (12,6)

df = pd.read_csv('baltimore_crime_2020_ts_041320pt.csv',
                 index_col=0,parse_dates=[0])
## Lazy fix to not changine col names to lowercase last class
df.columns = [col.lower() for col in df.columns]
df.index.freq = 'D'
display(df.head())

# df.index

In [ ]:
ax = df.plot()
ax.legend(bbox_to_anchor=([1,1]))
ax.set(title=f'Baltimore Crime Rates - {df.index.freq}')

# Trends/Stationarity

<div style="text-align:center;font-size:2em">Trends</div>
<img src="https://raw.githubusercontent.com/learn-co-students/dsc-removing-trends-online-ds-ft-100719/master/images/new_trendseasonal.png" width=80%>

<div style="text-align:center;font-size:2em">Mean</div>
    
<img src="https://raw.githubusercontent.com/jirvingphd/dsc-types-of-trends-online-ds-ft-100719/master/images/new_mean_nonstationary.png" width=70%>
<br><br>
<div style="text-align:center;font-size:2em">Variance</div>
<img src="https://raw.githubusercontent.com/jirvingphd/dsc-types-of-trends-online-ds-ft-100719/master/images/new_cov_nonstationary.png" width=70%>
</div>

### Testing for Stationarity

In [ ]:
from statsmodels.tsa.stattools import adfuller

def stationarity_check(TS,plot=True,col=None):
    """From: https://learn.co/tracks/data-science-career-v2/module-4-a-complete-data-science-project-using-multiple-regression/working-with-time-series-data/time-series-decomposition
    """
    
    # Import adfuller
    from statsmodels.tsa.stattools import adfuller

    if col is not None:
        # Perform the Dickey Fuller Test
        dftest = adfuller_test_df(TS[col])#adfuller(TS[col]) # change the passengers column as required 
    else:
        dftest=adfuller_test_df(TS)#adfuller(TS)
 
    if plot:
        # Calculate rolling statistics
        rolmean = TS.rolling(window = 8, center = False).mean()
        rolstd = TS.rolling(window = 8, center = False).std()

        #Plot rolling statistics:
        fig = plt.figure(figsize=(12,6))
        orig = plt.plot(TS, color='blue',label='Original')
        mean = plt.plot(rolmean, color='red', label='Rolling Mean')
        std = plt.plot(rolstd, color='black', label = 'Rolling Std')
        plt.legend(loc='best')
        plt.title('Rolling Mean & Standard Deviation')
        plt.show(block=False)
    display(dftest)


## Simpler Version of ADfullter func
def adfuller_test_df(ts):
    """Returns the AD Fuller Test Results and p-values for the null hypothesis
    that there the data is non-stationary (that there is a unit root in the data)"""
    df_res = adfuller(ts)
    names = ['Test Statistic','p-value','#Lags Used','# of Observations Used']
    res  = dict(zip(names,df_res[:4]))
    res['p<.05'] = res['p-value']<.05
    res['Stationary?'] = res['p<.05']
    
    return pd.DataFrame(res,index=['AD Fuller Results'])

# stationarity_check(ts);
# adfuller_test_df(ts)


In [ ]:
ts = df['larceny'].loc['2017':]
stationarity_check(ts);

## Removing Trends 

#### Trend Removal Methods
- Differencing (`.diff()`)
- Log-Transformation (`np.log`)
- Subtract Rolling Mean (`ts-ts.rolling().mean()`)
- Subtract Exponentially-Weighted Mean (`ts-ts.ewm().mean()`)
- Seasonal Decomposition (`from statsmodels.tsa.seasonal import seasonal_decompose`
)

In [ ]:
## Differencing 
ts0 = ts.diff().dropna()
ts0.plot()
adfuller_test_df(ts0)

In [ ]:
## Log Transform
ts3 = np.log(ts)
ts3.plot()
adfuller_test_df(ts3)

In [ ]:
## Subtract Rolling mean
ts2 = (ts - ts.rolling(3).mean()).dropna()
ts2.plot()
adfuller_test_df(ts2)

In [ ]:
## Subtract Exponentially Weight Mean Rolling mean
ts4 = (ts - ts.ewm(halflife=7).mean()).dropna()
ts4.plot()
adfuller_test_df(ts4)

## Seasonal Decomposition

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
plt.figure(figsize=(12,15))
decomp= seasonal_decompose(ts)#,model='mul')
decomp.plot();

# Sect 38: TIME SERIES MODELS

## Basic Time Series models

- White Noise Model
- Randon Walk Model

In [ ]:
## Load in example time series
plt.rcParams['figure.figsize'] = [12,5]
nyse = fs.datasets.load_ts_nyse_monthly(read_csv_kwds={'parse_dates':True,
                                                     'index_col':'Month'})
exch = fs.datasets.load_ts_exch_rates(read_csv_kwds={'parse_dates':['Frequency'],
                                                     'index_col':'Frequency'})
trends = fs.datasets.load_ts_google_trends(read_csv_kwds={'skiprows':1,
                                                         'parse_dates':['Month'],
                                                         'index_col':['Month']})

trends.columns= ['diet','gym','finance']

## White Noise Model

- White noise is a true stationary time series.
- 3 Properties:
    - Fixed and constant mean
    - Fixed and constant variance
    - No correlation over time

- Gaussian White Noise: A special case of a White Noise model is 
    - Mean is equal to zero
    - variance is equal to 1
    

In [ ]:
ax = nyse.plot(title='Example White Noise Model')

## Random Walk Model
- Two Properties:
    - Has no specified mean or variance
    - Has a strong dependence over time


- The changes over time are basically a white noise model. 

$$\large Y_t = Y_{t-1} + \epsilon_t$$

- Where $\epsilon_t$ is a *mean zero* white noise model!

In [ ]:
exch['Euro'].plot(subplots=True,figsize=(12,4),title='Example Random Walk Model');

### Random Walk with a Drift:
- a drift parameter $c$, steering in a certain direction.
$$\large Y_t = c + Y_{t-1} + \epsilon_t$$

- When a random walk is differenced it returns a white noise: 


In [ ]:
exch['Euro'].diff().plot(subplots=True,figsize=(12,4),
                         title='Differenced Random Walks Become White Noise');

# Correlation, Autocorrelation & Partial Autocorrelation

In [ ]:
trends.plot()

In [ ]:
## Correlation of Raw TS
trends.corr().style.background_gradient()

> #### Detrending Reveals Higher Cross-Correlation

In [ ]:
trends.diff().plot()

In [ ]:
## We Removed the Trend without Removing Seaonality
trends.diff().corr().style.background_gradient()

In [ ]:
## Seasonality causes correlations at specific times
n=12
ts = trends[['diet']]
ts_shifted = ts.loc['2011':'2016']
ts_shifted[f"Shifted by {n}"] = ts_shifted.shift(n)
ts_shifted.plot()

### Demonstrating Autocorrelation

In [ ]:
ts = trends['diet']
ts.plot()

In [ ]:
## Generate 6 time-shifted columns
total_shifts =6
shifts = [ts.shift(x).rename(f"Diet shifted {x}") for x in range(total_shifts)]
res = pd.concat(shifts,axis=1)
res

In [ ]:
res.plot()

In [ ]:
res.corr()[['Diet shifted 0']].style.background_gradient()

In [ ]:
## With 160 shifts
total_shifts = 160
shifts = [ts.shift(x).rename(f"Diet shifted {x}") for x in range(total_shifts)]
res = pd.concat(shifts,axis=1)
res

In [ ]:
res_corr = res.corr()[['Diet shifted 0']]
display(res_corr.iloc[:24].style.background_gradient())

In [ ]:
res_corr.plot(figsize=(12,3))

## ACF & PACF  Plots

### Autocorrelation Function Plots

https://machinelearningmastery.com/gentle-introduction-autocorrelation-partial-autocorrelation/

> - "The **autocorrelation function** is a function that represents autocorrelation of a time series as a function of the time lag."
- The autocorrelation function tells interesting stories about trends and seasonality. For example, if the original time series repeats itself every five days, you would expect to see a spike in the autocorrelation function at 5 days.



In [ ]:
plt.rcParams['figure.figsize'] = (12,4)
ts = trends['diet']
ts.plot()

In [ ]:
pd.plotting.autocorrelation_plot(ts)

In [ ]:
import statsmodels.graphics.tsaplots as tsa

tsa.plot_acf(ts);

In [ ]:
## Plot differenced data
ts_diff = ts.diff().dropna()
ts_diff.plot()

In [ ]:
pd.plotting.autocorrelation_plot(ts_diff)

In [ ]:
tsa.plot_acf(ts_diff);

### Partial-Autocorrelation Function Plot


> "The **partial autocorrelation function** can be interpreted as a regression of the series against its past lags.
 
 > It helps you come up with a possible order for the auto regressive term. The terms can be interpreted the same way as a standard linear regression, that is the contribution of a change in that particular lag while holding others constant. "

In [ ]:
tsa.plot_pacf(ts);

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
plot_acf(ts);
plot_pacf(ts);

#  ARMA MODELS


## Autoregressive Model (AR)


$$ \text{Today = constant + slope} \times \text{yesterday + noise} $$

Or, mathematically:

$$\large Y_t = \mu + \phi * Y_{t-1}+\epsilon_t$$

> Some notes based on this formula:
- If the slope is 0, the time series is a white noise model with mean $\mu$
- If the slope is not 0, the time series is autocorrelated
- Bigger slope means bigger autocorrelation
- When there is a negative slope, the time series follows an oscillatory process
<img src="https://raw.githubusercontent.com/jirvingphd/dsc-arma-models-online-ds-pt-100719/master/images/AR_model.png" width=90%>


## The  Moving Average Model


- The weighted sum of today's and yesterday's noise.

In words, the mathematical idea is the following:

$$ \text{Today = Mean + Noise + Slope} \times \text{yesterday's noise} $$

Or, mathematically:
$$\large Y_t = \mu +\epsilon_t + \theta * \epsilon_{t-1}$$

> Some notes based on this formula:
- If the slope is 0, the time series is a white noise model with mean $\mu$
- If the slope is not 0, the time series is autocorrelated and depends on the previous white noise process
- Bigger slope means bigger autocorrelation
- When there is a negative slope, the time series follow an oscillatory process
<img src="https://raw.githubusercontent.com/learn-co-students/dsc-arma-models-onl01-dtsc-pt-041320/master/images/MA_model.png" width=90%>


## Higher-order AR and MA models




Let's look at the formulas of AR and MA again:

- AR: $Y_t = \mu + \phi * Y_{t-1}+\epsilon_t$
- MA: $Y_t = \mu +\epsilon_t + \theta * \epsilon_{t-1}$

Note that these models are constructed in a way that processes only depend directly on the previous observation in the process. These models are so-called "1st order models", and denoted by AR(1) and MA(1) processes respectively. Let's look at AR(2) and MA(2).

- AR(2): $$Y_t = \mu + \phi_1 * Y_{t-1}+\phi_2 * Y_{t-2}+\epsilon_t$$
- MA(2): $$Y_t = \mu +\epsilon_t + \theta_1 * \epsilon_{t-1}+ \theta_2 * \epsilon_{t-2}$$



## ARMA models

- Combination of AR & MA models.
    - regression on past values takes place (AR part) 
    - and also that the error term is modeled as a linear combination of error terms of the recent past (MA part).
  
  
  
- **Generally, one denotes ARMA as ARMA(p, q).**


- An ARMA(2,1) model is given by:

 $$Y_t = \mu + \phi_1 Y_{t-1}+\phi_2 Y_{t-2}+ \theta \epsilon_{t-1}+\epsilon_t$$




___

### DETERMINING AR(P) and MA(Q) FROM READING PACF/ACF 

#### INFO FROM LESSONS:

- AR(p):
    - ACF for AR(p) would be strong until lag of p, then stagnant, then trail off. 
    - PACF for AR(p): Generally no correlation for lag values beyond p.
- MA(q):
    - ACF for MA(q) would show strong correlation up to a lag of q, the immedately delcine to minimal/no correction.
    - PACF would show strong relationship to the lag and tailing off to no correlation afterwards.
   

| Param| AR(p)   |   MA(q)  | ARMA(p,q)|
|------|------|------|------|
|   ACF | Tails off   |  Cuts off after lag q |  Tails off   |
|   PACF | Cuts off after lag p  |   Tails off  |  Tails off  |
 
 
#### INFO FROM UDEMY

- **USE ACF TO JUDGE IF MA OR AR COMPONENTS:**
    - If lag 1 is positive: AR
    - If lag 1 is negatige: MA
    
- **PACF is best for picking AR (p)**
- **ACF is best for picking MA(q)**
    - If sharp drop off at lag of k (k= point on x axis) means use an AR model of order k.
    - If slow gradual decline: use MA
   

# ARIMA MODELS:


## The ARIMA Time Series Model

One of the most common methods used in time series forecasting is known as the ARIMA model, which stands for **AutoregRessive Integrated Moving Average**. ARIMA is a model that can be fitted to time series data in order to better understand or predict future points in the series.

Let's have a quick introduction to ARIMA. The ARIMA forecasting for a stationary time series is nothing but a linear (like a linear regression) equation. The predictors depend on the parameters (p,d,q) of the ARIMA model:

### Number of AR (Auto-Regressive) terms (p): 

`p` is the auto-regressive part of the model. It allows us to incorporate the effect of past values into our model. Intuitively, this would be similar to stating that it is likely to rain tomorrow if it has been raining for past 3 days. AR terms are just lags of dependent variable. For instance if p is 5, the predictors for x(t) will be x(t-1)….x(t-5).

### Number of Differences (d):

`d` is the **Integrated** component of an ARIMA model. This value is concerned with the amount of differencing as it identifies the number of lag values to subtract from the current observation. Intuitively, this would be similar to stating that it is likely to rain tomorrow if the difference in amount of rain in the last *n* days is small. 

### Number of MA (Moving Average) terms (q): 

`q` is the moving average part of the model which is used to set the error of the model as a linear combination of the error values observed at previous time points in the past. MA terms form lagged forecast errors in prediction equation. For instance if q is 5, the predictors for x(t) will be e(t-1)….e(t-5) where `e(i)` is the difference between the moving average at ith instant and actual value.

These three distinct integer values, (p, d, q), are used to parametrize ARIMA models. Because of that, ARIMA models are denoted with the notation `ARIMA(p, d, q)`. Together these three parameters account for seasonality, trend, and noise in datasets:

* `(p, d, q)` are the non-seasonal parameters described above.
* `(P, D, Q)` follow the same definition but are applied to the seasonal component of the time series. 
* The term `s` is the periodicity of the time series (4 for quarterly periods, 12 for yearly periods, etc.).

A detailed article on these parameters is available [HERE](https://www.quantstart.com/articles/Autoregressive-Integrated-Moving-Average-ARIMA-p-d-q-Models-for-Time-Series-Analysis).

The seasonal ARIMA method can appear daunting because of the multiple tuning parameters involved. In the next section, we will describe how to automate the process of identifying the optimal set of parameters for the seasonal ARIMA time series model.

# Activity: SARIMA Lab

- Repo folder > labs from class> sect_38

# Appendix

In [ ]:
plot_acf(ts);
plot_pacf(ts);

## `pmdarima.auto_arima`

In [ ]:
# !pip install -U pmdarima
import pmdarima
pmdarima.auto_arima?
